# Spam Detection System

## Importing Libraries

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.\
        builder.\
        appName('Spam_detection').\
        getOrCreate()

## Preparing the data

### Importing data

In [3]:
df = spark.read.csv('SMSSpamCollection', inferSchema=True, sep='\t').\
        withColumnRenamed('_c0', 'class').\
        withColumnRenamed('_c1', 'text')

In [4]:
df.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [5]:
from pyspark.sql.functions import length
df = df.withColumn('length', length(df['text']))
df.show(5)

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
+-----+--------------------+------+
only showing top 5 rows



In [6]:
df.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



### Processing the Text

In [7]:
from pyspark.ml.feature import (Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer)

In [8]:
tokenizer = Tokenizer(inputCol = 'text', outputCol = 'token_text')
stop_remover = StopWordsRemover(inputCol = 'token_text', outputCol='stop_token')
count_vec = CountVectorizer(inputCol = 'stop_token', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol = 'class', outputCol='label')

In [9]:
from pyspark.ml.feature import VectorAssembler
clean_up = VectorAssembler(inputCols = ['tf_idf', 'length'], outputCol='features')

In [10]:
from pyspark.ml import Pipeline
pipe = Pipeline(stages = [ham_spam_to_numeric, tokenizer,
                         stop_remover, count_vec, idf, clean_up]).fit(df)

In [11]:
pdf = pipe.transform(df)
pdf = pdf.select('label', 'features')
pdf.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
+-----+--------------------+
only showing top 5 rows



In [12]:
train_data, test_data = pdf.randomSplit([0.7, 0.3])

In [13]:
from pyspark.ml.classification import NaiveBayes
spam_detector = NaiveBayes().fit(train_data)
results = spam_detector.transform(test_data)
results.select('label', 'prediction').show()

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
+-----+----------+
only showing top 20 rows



In [14]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
accuracy = MulticlassClassificationEvaluator(metricName='accuracy')
print('Accuracy of NB Model: {:.3f}'.format(accuracy.evaluate(results)))

Accuracy of NB Model: 0.919
